| NLP | RAG Ollama | Auto Merging Retrieval |

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
%%capture
!pip install trulens-eval
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install torch sentence-transformers
!pip install llama-index-llms-ollama
!pip install ipython ipywidgets
!pip install litellm
!pip install langchain
!pip uninstall -y trulens_eval
!pip install trulens_eval


In [2]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["xxx.pdf"]
).load_data()

In [3]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 



41 



<class 'llama_index.core.schema.Document'>

Doc ID: 696a7571-366d-48f7-8091-c3569f014b4c

Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng

How to  Build Your Career in AIA Simple Guide


## Auto-merging retrieval setup

In [4]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [5]:
from llama_index.core.node_parser import HierarchicalNodeParser

# create the hierarchical node parser w/ default settings
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)

In [6]:
nodes = node_parser.get_nodes_from_documents([document])

In [7]:
from llama_index.core.node_parser import get_leaf_nodes

leaf_nodes = get_leaf_nodes(nodes)
print(leaf_nodes[2].text)

Chapter 11: Overcoming Imposter Syndrome.

Final Thoughts: Make Every Day Count.LEARNING

PROJECTS

JOB



PAGE 4Coding AI Is the New Literacy

Today we take it for granted that many people know how to read and write. Someday, I hope, 

it will be just as common that people know how to write code, specifically for AI.

Several hundred years ago, society didn’t view language literacy as a necessary skill. A small 

number of people learned to read and write, and everyone else let them do the reading and 

writing.


In [8]:
nodes_by_id = {node.node_id: node for node in nodes}

parent_node = nodes_by_id[leaf_nodes[2].parent_node.node_id]
print(parent_node.text)

PAGE 1Founder, DeepLearning.AICollected Insights

from Andrew Ng

How to 

Build

Your

Career

in AIA Simple Guide





PAGE 2"AI is the new 

electricity. It will 

transform and improve 

all areas of human life."

Andrew Ng



PAGE 3Table of 

ContentsIntroduction: Coding AI is the New Literacy.

Chapter 1: Three Steps to Career Growth.

Chapter 2: Learning Technical Skills for a 

Promising AI Career.

Chapter 3: Should You Learn Math to Get a Job 

in AI?

Chapter 4: Scoping Successful AI Projects.

Chapter 5: Finding Projects that Complement 

Your Career Goals.

Chapter 6: Building a Portfolio of Projects that 

Shows Skill Progression.

Chapter 7: A Simple Framework for Starting Your AI 

Job Search.

Chapter 8: Using Informational Interviews to Find 

the Right Job.

Chapter 9: Finding the Right AI Job for You.

Chapter 10: Keys to Building a Career in AI.

Chapter 11: Overcoming Imposter Syndrome.

Final Thoughts: Make Every Day Count.LEARNING

PROJECTS

JOB



PAGE 4Coding 

## Putting it all Together

In [9]:
import os


from llama_index.core import (
    ServiceContext,
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.llms.ollama import Ollama
from llama_index.core.node_parser import HierarchicalNodeParser
from llama_index.core.node_parser import get_leaf_nodes
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core.query_engine import RetrieverQueryEngine



def build_automerging_index(
    documents,
    llm = Ollama(model="mistral", temperature=0.1, request_timeout=300.0),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)
    merging_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
    )
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes, storage_context=storage_context, service_context=merging_context
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=merging_context,
        )
    return automerging_index


def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    rerank_top_n=6,
    llm = Ollama(model="mistral", temperature=0.1, request_timeout=300.0),
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank], llm=llm
    )
    return auto_merging_engine

In [10]:
llm = Ollama(model="mistral", temperature=0.1, request_timeout=500.0)
index = build_automerging_index(
    [document],
    llm=llm,
    save_dir="./merging_index",
)


In [11]:
query_engine = get_automerging_query_engine(index, similarity_top_k=6)

In [12]:
from llama_index.core.chat_engine.condense_question import (
    CondenseQuestionChatEngine,
)

In [13]:
automerging_chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine, llm=llm
)

In [14]:
def chat(message):
    res = automerging_chat_engine.stream_chat(message)
    for text in res.response_gen:
        yield text

In [15]:
for part in chat("What is the importance of networking in AI?"):
    print(part, end='')

 Networking plays a crucial role in the professional growth of individuals in the field of Artificial Intelligence (AI). It provides opportunities to build genuine relationships with like-minded professionals who can offer valuable information, advice, and support. These connections can be particularly helpful during challenging moments in one's career. Furthermore, networking can lead to referrals for potential job opportunities. Despite the initial discomfort or reluctance some individuals may have towards networking events, it is essential for expanding one's professional network and advancing within the AI community.

## TruLens Evaluation

In [16]:
from trulens_eval import Tru

Tru().reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .

🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


### Two layers

In [17]:
llm = Ollama(model="mistral", temperature=0.1, request_timeout=300.0)
auto_merging_index_0 = build_automerging_index(
        [document],
        embed_model="local:BAAI/bge-small-en-v1.5",
        llm=llm,
        chunk_sizes=[2048,512],
        save_dir="./merging_index_0",
)


In [18]:
auto_merging_engine_0 = get_automerging_query_engine(
    auto_merging_index_0,
    similarity_top_k=12,
    rerank_top_n=6,
)

In [19]:
def get_prebuilt_trulens_recorder(query_engine, app_id):
    from trulens_eval.feedback.provider.litellm import LiteLLM
    from trulens_eval import (
            Feedback,
            TruLlama,
            OpenAI
        )
    from trulens_eval.feedback import Groundedness
    import numpy as np
    from litellm import completion
   
    llm = LiteLLM("ollama/mistral", completion_kwargs={
        "api_base": "http://localhost:11434"
    })
    
    qa_relevance = (
        Feedback(llm.relevance_with_cot_reasons, name="Answer Relevance")
        .on_input_output()
    )

    qs_relevance = (
        Feedback(llm.relevance_with_cot_reasons, name = "Context Relevance")
        .on_input()
        .on(TruLlama.select_source_nodes().node.text)
        .aggregate(np.mean)
    )

    #  grounded = Groundedness(groundedness_provider=openai, summarize_provider=openai)
    grounded = Groundedness(groundedness_provider=llm)

    groundedness = (
        Feedback(grounded.groundedness_measure_with_cot_reasons, name="Groundedness")
            .on(TruLlama.select_source_nodes().node.text)
            .on_output()
            .aggregate(grounded.grounded_statements_aggregator)
    )

    feedbacks = [qa_relevance, qs_relevance, groundedness]
    tru_recorder = TruLlama(
        query_engine,
        app_id=app_id,
        feedbacks=feedbacks
    )
    return tru_recorder

In [20]:

tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_0,
    app_id ='app_0'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .

✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .

✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .

✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .

✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [25]:
questions = [
    "What is the importance of networking in AI?",
    "Explain the scenario planning exercise mentioned in the context of the Covid-19 pandemic. How can this method be applied to personal life situations such as job hunting or exam preparation?",
    "In the context of project selection and execution, explain the difference between the 'Ready, Aim, Fire' and 'Ready, Fire, Aim' approaches. Provide examples where each approach might be more beneficial.",
]


In [26]:
def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

In [27]:
run_evals(questions, tru_recorder, auto_merging_engine_0)

> Filling in node. Node id: a9f10c50-f372-440a-b2e9-18f83318f6e0> Node text: Chapters with the 

cover topics about 

learning foundational 

technical skills. After having gain...



> Merging 3 nodes into parent node.

> Parent node id: 5737eb7a-94bd-447e-a32e-13e6f27b3c62.

> Parent node text: I’ve known many machine learning engineers who benefitted from 

deeper skills in an application a...



> Merging 5 nodes into parent node.

> Parent node id: 59b40fc6-391d-4065-b3d0-5917701d9129.

> Parent node text: PAGE 1Founder, DeepLearning.AICollected Insights

from Andrew Ng

How to 

Build

Your

Career

in AIA ...



> Merging 3 nodes into parent node.

> Parent node id: 1c9b7338-c90b-47ff-921f-95a8c07ddb6b.

> Parent node text: In this chapter, I’d like to focus on a framework that’s useful for many job seekers in 

AI, espe...



> Merging 3 nodes into parent node.

> Parent node id: 158a9917-100c-4270-aaba-1aea71076cca.

> Parent node text: If you find someone else with an idea, a

In [28]:
from trulens_eval import Tru

Tru().get_leaderboard(app_ids=[])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
app_0,0.785,0.653333,1.0,92.166667,0.0


In [29]:
Tru().run_dashboard()

Starting dashboard ...

Config file already exists. Skipping writing process.

Credentials file already exists. Skipping writing process.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...

To disable this warning, you can either:

	- Avoid using `tokenizers` before the fork if possible

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.0.84:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### Three layers

In [30]:
auto_merging_index_1 = build_automerging_index(
        [document],
        embed_model="local:BAAI/bge-small-en-v1.5",
        llm=llm,
        chunk_sizes=[2048,512, 128],
        save_dir="./merging_index_0",
)


In [31]:
auto_merging_engine_1 = get_automerging_query_engine(
    auto_merging_index_1,
    similarity_top_k=12,
    rerank_top_n=6,
)


In [32]:
tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_1,
    app_id ='app_1'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .

✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .

✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .

✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .

✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [33]:
run_evals(questions, tru_recorder, auto_merging_engine_1)

> Filling in node. Node id: a9f10c50-f372-440a-b2e9-18f83318f6e0> Node text: Chapters with the 

cover topics about 

learning foundational 

technical skills. After having gain...



> Merging 3 nodes into parent node.

> Parent node id: 5737eb7a-94bd-447e-a32e-13e6f27b3c62.

> Parent node text: I’ve known many machine learning engineers who benefitted from 

deeper skills in an application a...



> Merging 5 nodes into parent node.

> Parent node id: 59b40fc6-391d-4065-b3d0-5917701d9129.

> Parent node text: PAGE 1Founder, DeepLearning.AICollected Insights

from Andrew Ng

How to 

Build

Your

Career

in AIA ...



> Merging 3 nodes into parent node.

> Parent node id: 1c9b7338-c90b-47ff-921f-95a8c07ddb6b.

> Parent node text: In this chapter, I’d like to focus on a framework that’s useful for many job seekers in 

AI, espe...



> Merging 3 nodes into parent node.

> Parent node id: 158a9917-100c-4270-aaba-1aea71076cca.

> Parent node text: If you find someone else with an idea, a

In [ ]:
from trulens_eval import Tru

Tru().get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
app_0,0.666667,1.0,1.0,8.666667,0.0
app_1,0.666667,1.0,1.0,8.666667,0.0


In [ ]:
Tru().run_dashboard()

Starting dashboard ...

Config file already exists. Skipping writing process.

Credentials file already exists. Skipping writing process.

Dashboard already running at path:   Network URL: http://192.168.0.84:8501




<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>